# Planet Quota API Introduction

This notebook serves as a demo of the basic use and functionality of [Planet's](https://www.planet.com) Quota Reservations API. The Quota API will allow you to create, estimate, and view existing quota reservations on the Planet platform for select products that require Subscriptions to receive. This tutorial assumes familiarity with the [Python](https://python.org) programming language, as well as basic [REST API](https://restfulapi.net/) concepts.

## Requirements

- An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's APIs.
    - This demo will handle authentication for the Quota API with an API key.
    - Instructions on obtaining your Planet API key can be found in the [API authentication documentation](https://docs.planet.com/develop/authentication/#obtaining-your-api-key).
- AOI Features on the Planet Platform, created with the Planet [Features API](https://docs.planet.com/develop/apis/features/) or Features Manager.
- Access to supported products that require Quota Reservations.

## Supported Products

The following products have quotas that can be managed by the Quota API:

- [Analysis Ready PlanetScope (ARPS)](https://docs.planet.com/data/imagery/arps/)
- [Planetary Variables](https://docs.planet.com/data/planetary-variables/)
- [PlanetScope](https://docs.planet.com/data/imagery/planetscope/) 
    - Select PlanetScope monitoring and access products, but these will be discussed with you by a CSM.


## Useful links 
* [Planet Quota API Documentation](https://docs.planet.com/develop/apis/quota/)
* [Planet Features API Documentation](https://docs.planet.com/develop/apis/features/)

____

## Setup

In order to interact with the Planet Quota API, we need to import the necessary packages and authenticate our session with our API Key.

### Imports

In [1]:
import requests
import os

# Setup Planet Quota API base URL
QUOTA_URL = "https://api.planet.com/account/v1/quota-reservations/"

### Authentication

Authenticating your session for API requests will use your API key.

You should *export* your API Key as an environment variable on your system:
```bash
export PL_API_KEY="YOUR API KEY HERE"
```
Or add the variable to your path, etc.

In [2]:
# Authenticate your session using your API key; See docs: https://docs.planet.com/develop/authentication
# Check for PL_API_KEY environment variable, otherwise type in your API key.
pl_api_key = os.getenv('PL_API_KEY')
if not pl_api_key:
    import getpass
    pl_api_key = getpass.getpass('Planet API Key: ')
    os.environ['PL_API_KEY'] = pl_api_key

# Setup and authenticate the session
session = requests.Session()
session.auth = (pl_api_key, '')

_____

### Basic Quota API Functionality

The most basic functions of the Quota API are to check for existing quota reservations and the products you have available quota for. 

First, check for any existing quota reservations you might have. If you have not actually made a quota reservation yet, the `count` key will be zero in the results.

In [ ]:
# Successful requests will print `Status code: 200`

quota_res = session.get(QUOTA_URL)
print("Status code:", quota_res.status_code)

Status code: 200


In [6]:
quota_res.json()

{'meta': {'args': {'limit': 50, 'offset': 0},
  'count': 0,
  'next': None,
  'prev': None},
 'results': []}

To check the products that you have available to make quota reservations for, you can send a GET request to the `/my/products/` endpoint. Below we will retrieve the product `id` from the results of this request to make our quota reservations.

While the product name of `PS_ARPS_SR_DAILY_ALL_TIME`, is used for all Analysis Ready PlanetScope Subscriptions, the `id` will be a unique integer for you, as the assignment of the product id is by both the product and the account given access to it.
<br><br>

The important keys from products endpoint include:

- `id`: The unique identifier of how this product is assigned to you, to use as product_id in reservation requests
- `name`: The product identifier, independent of assignment (e.g., "FCM_30M")
- `title`: Human-readable product name
- `description`: Product description
- `supports_reservation`: Boolean indicating if the product supports quota reservations (only products with true can be used with reservation endpoints)
- `quota_total`: Total quota available for your organization
- `quota_used`: Quota already consumed or reserved
- `unlimited_quota`: Whether the product has unlimited quota


Note that if you're looking for subscriptions to products that don't require quota reservations (such as regular PlanetScope subscriptions), use the `/my/subscriptions/` endpoint instead. Note that this endpoint will return an error if you do not have access to those subscriptions in your plan.





In [5]:
# products endpoint url
PRODUCTS_URL = "https://api.planet.com/account/v1/my/products/"

products_response = session.get(PRODUCTS_URL)
products_response.raise_for_status()

In [8]:
products_response.json()

{'meta': {'args': {'limit': 50, 'offset': 0},
  'count': 1,
  'next': None,
  'prev': None},
 'results': [{'area': {'description': 'Minimum AOI spend.',
    'op': 'gte',
    'units': 'sqkm',
    'value': 25},
   'billable_id': 'prn:usage-api:bucket:8aa442a6-5838-4289-89e6-8e215cdfbc83',
   'created_at': '2026-01-26T18:37:57.356369',
   'description': 'Daily pre-processed, surface reflectance data prepared for analysis. All Time Archive.',
   'id': 290378,
   'name': 'PS_ARPS_SR_DAILY_ALL_TIME',
   'organization_id': 776979,
   'product_family': 'Planet Analysis Ready Data (L3)',
   'quota_total': 100000.0,
   'quota_units': 'sqkm',
   'quota_used': 0.0,
   'resource_ids': ['PS_ARD_SR_DAILY'],
   'state': 'active',
   'supports_reservation': True,
   'term_end_date': '2027-01-01T23:59:59.999999',
   'term_start_date': '2026-01-01T00:00:00',
   'title': 'Surface Reflectance ARD - Daily (All Time Archive)',
   'unlimited_quota': False,
   'updated_at': '2026-01-26T18:37:57.356377'}]}

In [ ]:
# If you have access to multiple products,
# you can change the index number here to get the desired product ID
# The demo will just use the first product in the list.
product_response_json = products_response.json()['results'][0]

product_id = product_response_json['id']

___

### Features for Quota Reservations

When we submit requests to the Quota API, we will need a Feature Reference ID OR a Collection ID to link our request to an individual or set of Features within a Feature Collection on the Planet Platform. If you do not have any Features currently, you can learn how to use the Planet Python SDK to upload them with the [Planet Python SDK Features Demo](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/api_guides/features_api/planet_sdk_features_demo.ipynb) notebook, or use the `requests` module based [Planet Features API](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/api_guides/features_api/planet_features_api.ipynb) notebook.

The code below will assume you have Features already created, and makes two requests to the Features API. The first will get our Feature Collections, which will include a url for the collections themselves, as well as the reference ID of the Collection. The second will get a Collection from its URL, which will return a list of its Features, which will all have Feature Reference IDs. _Both the Collection Reference IDs and Features Reference IDs are called `'pl:ref'` in the keys of the request return, so we will save them to different variables to avoid confusion._  
<br><br>
If you already have a Feature Reference ID OR a Collection ID handy for a Feature or Collection more relevant to you, feel free to set it as the pl_ref variable and skip the calls to the Feature API.

In [ ]:
FEATURE_URL = "https://api.planet.com/features/v0/ogc/my/collections"

collections_response = session.get(FEATURE_URL)
collections_response.raise_for_status()
collections_json = collections_response.json()

Status code: 200


In [153]:
collection_ref_id = collections_json['collections'][0]['pl:ref']

# The first URL link will return the entire Collection,
# which we will use to get the list of Features in the Collection.
collection_items_url = collections_json['collections'][0]['links'][1]['href']
print(f"Collection ref ID: {collection_ref_id}, Collection Items URL: {collection_items_url}, use this URL to get the Feature Reference IDs of Features in the collection.")

Collection ref ID: pl:features/my/cairo_areas-Kr7JYmq, Collection Items URL: https://api.planet.com/features/v0/ogc/my/collections/cairo_areas-Kr7JYmq/items, use this URL to get the Feature Reference IDs of Features in the collection.


In [148]:
collections_json

{'numberMatched': 1,
 'links': [{'href': 'https://api.planet.com/features/v0/ogc/my/collections',
   'rel': 'self',
   'title': 'This page of results'}],
 'collections': [{'id': 'cairo_areas-Kr7JYmq',
   'title': 'cairo_areas',
   'description': 'FeatureCollection of two areas in Cairo, Egypt',
   'item_type': 'feature',
   'created': '2025-12-19T23:22:29Z',
   'updated': '2025-12-19T23:22:33Z',
   'extent': {'spatial': {'bbox': [[31.23344423,
       29.99579775,
       31.26630889,
       30.06461941]]}},
   'links': [{'href': 'https://api.planet.com/features/v0/ogc/my/collections/cairo_areas-Kr7JYmq',
     'rel': 'self',
     'title': 'This collection',
     'type': 'application/json'},
    {'href': 'https://api.planet.com/features/v0/ogc/my/collections/cairo_areas-Kr7JYmq/items',
     'rel': 'features',
     'title': 'Features',
     'type': 'application/json'}],
   'feature_count': 2,
   'area': 9890943.929543883,
   'title_property': None,
   'description_property': None,
   'prop

In [47]:
# We will use the first item in the collection for this example

collection_items = session.get(collection_items_url)
pl_ref = collection_items.json()['features'][0]['properties']['pl:ref']

In [48]:
collection_items.json()

{'type': 'FeatureCollection',
 'numberMatched': 2,
 'links': [{'href': 'https://api.planet.com/features/v0/ogc/my/collections/cairo_areas-Kr7JYmq/items',
   'rel': 'self',
   'title': 'This page of results'}],
 'features': [{'type': 'Feature',
   'id': '1',
   'properties': {'title': 'institute_area',
    'name_str': 'area_2',
    'extra_property_1': 'extra property 1 here',
    'extra_property_2': 'extra property 2 here',
    'pl:ref': 'pl:features/my/cairo_areas-Kr7JYmq/1-owL5RyP',
    'pl:area': 3104292.7715780144},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[31.23344423, 30.05109414],
      [31.25491533, 30.05109414],
      [31.25491533, 30.06461941],
      [31.23344423, 30.06461941],
      [31.23344423, 30.05109414]]]}},
  {'type': 'Feature',
   'id': '0',
   'properties': {'title': 'lake_area',
    'name_str': 'area_1',
    'extra_property_1': 'extra property 1 here',
    'extra_property_2': 'extra property 2 here',
    'pl:ref': 'pl:features/my/cairo_areas-Kr7JYmq/0

In [ ]:
# Use a Collection Reference ID you want to reserve quota for a whole collection of features

collection_ref_id

'pl:features/my/cairo_areas-Kr7JYmq'

In [50]:
# Use one or more Feature Reference IDs in a list to reserve quota for specific features

pl_ref

'pl:features/my/cairo_areas-Kr7JYmq/1-owL5RyP'

___

### Quota Reservation Estimates

Now that we have a Feature Reference ID handy, we will first post a request to the `/estimate/` endpoint of the Quota API to calculate a `total_cost` for a quota reservation. The only required parameters for your reservation estimate are either a list of Feature Reference ID(s) _or_ a single Collection ID, with a product ID. We set the variable `product_id` above. You will receive an error if you try to use both Feature Reference ID(s) _or_ a single Collection ID.

____
*IMPORTANT* Quota Reservation post requests will return a `quota_remaining` key, which is your quota remaining at the time of the post request, _NOT_ what will be remaining from use of the estimate. You have to do the math yourself.
____
*Note* If you use a Feature Reference ID and not a Collection Reference or vice versa, you cannot have the other in the payload as None, as this will return an error.

In [ ]:
ESTIMATE_URL = QUOTA_URL + 'estimate'

# aoi_refs_for_estimate input should be a list of pl:refs
aoi_refs_for_estimate = [pl_ref]

# If you have an alternative Collection than the one the example used above, paste it here.
# collection_ref_id = None

In [ ]:
estimate_payload = {
    "aoi_refs": aoi_refs_for_estimate,
    "product_id": product_id,
}

estimate_response = session.post(ESTIMATE_URL, json=estimate_payload)
estimate_res_json = estimate_response.json()

In [155]:
estimate_res_json

{'estimated_costs': [{'aoi_ref': 'pl:features/my/cairo_areas-Kr7JYmq/1-owL5RyP',
   'cost': 25.0,
   'reserved': True}],
 'quota_remaining': 99950.0,
 'quota_total': 100000.0,
 'quota_units': 'sqkm',
 'total_cost': 25.0}

In [90]:
collection_ref_id

'pl:features/my/cairo_areas-Kr7JYmq'

____

### Submitting Quota Reservations or Bulk Quota Reservations

You can create a Quota Reservation with a POST request to the main Quota API URL. Please use the `/bulk-reserve/` endpoint to asynchronously make bulk requests for many reservations with many different features.

The Quota Reservation payload to post your request is identical to the one we used above for our estimate.
____

## ****WARNING****: Following this demo will consume quota when you submit your reservation! 
## Please be _very_ sure that the quota reservation you are submitting is correct! 
### You will need to contact your CSM to get quota released after it has been submitted, even if you have made a subscription and recieved imagery yet.

##### Quota Reservation by Feature Reference IDs

In [ ]:
# aoi_refs_for_reservation input should be a list of pl:refs
# It is commented out so you have to be intentional about what you reserve quota for.

# aoi_refs_for_reservation = []


quota_reservation_payload = {
    "aoi_refs": aoi_refs_for_reservation,
    "product_id": product_id,
}

In [ ]:
#### Feature Reference IDs based Quota Reservation POST

reservation_response = session.post(QUOTA_URL, json=quota_reservation_payload)
reservation_response.raise_for_status()
reservation_response_json = reservation_response.json()

In [ ]:
reservation_response_json

{'quota_remaining': '99975000000.0',
 'quota_reservations': [{'aoi_ref': 'pl:features/my/cairo_areas-Kr7JYmq/1-owL5RyP',
   'id': 1413740,
   'state': 'reserved'}],
 'quota_total': 100000000000.0,
 'quota_units': 'sqm',
 'quota_used': 25000000.0,
 'url': 'https://api.planet.com/auth/v1/experimental/public/quota-reservations/'}

##### Bulk Quota Reservation Request


Bulk Quota Reservation Requests should be used when you have many Features in a collection (or a giant list) and you want to submit them all at once.

The `/bulk-reserve/` endpoint will be used, and for this example we will use a Collection ID in the payload instead of a list of Feature Reference IDs. All the Quota Reservation Requests for the Features will run asynchronously as a `Job`, which are trackable with a separate API call.

In [ ]:
# collection_id_for_reservation input should be a one Collection Reference ID
# It is commented out so you have to be intentional about what you reserve quota for

# collection_id_for_reservation =

# If you have a list of Feature Reference ID's, use the  "aoi_refs" key for your payload instead.
# aoi_refs_for_reservation = []

BULK_RESERVATION_URL = QUOTA_URL +  'bulk-reserve'


bulk_reservation_payload = {
    "collection_id": collection_id_for_reservation,
    "product_id": product_id
}

In [ ]:
#### BULK Quota Reservation POST

bulk_reservation_response = session.post(BULK_RESERVATION_URL, json=bulk_reservation_payload)
bulk_reservation_response.raise_for_status()
bulk_response_json = bulk_reservation_response.json()

Status code: 201


In [80]:
bulk_response_json

{'job_id': '8016', 'status': 'pending'}

Notice how this returns a `job_id` and a `status`. We will then use an additional endpoint, `/jobs/` with the `job_id` to check its status (again), check the `percentage` that the job is complete, along with other information. Note that you can list all the jobs for your organization with a GET request to just the `/jobs/` endpoint without a `job_id`.

In [ ]:
JOB_URL = QUOTA_URL + 'jobs/'

JOB_ID_URL = JOB_URL + bulk_response_json['job_id']

job_query_response = session.get(JOB_ID_URL)
job_query_response.raise_for_status()
job_query_response_json = job_query_response.json()

Status code: 200


In [82]:
job_query_response_json

{'collection_id': 'pl:features/my/cairo_areas-Kr7JYmq',
 'id': 8016,
 'organization_id': 776979,
 'percentage': 50.0,
 'processed_items': 1,
 'product_access_id': 290378,
 'status': 'completed',
 'total_items': 2,
 'type': 'quota:reserve'}

___

### Have you made a mistake and submit an incorrect Quota Reservation?

If you make a incorrect quota request, you must contact Planet Support or your assigned CSM. If you made an honest mistake here they can work with you to delete the quota reservation and releasing the quota back to your total.

_____

### Check our Quota Reservations Again

After you make a Quota or Bulk Quota Reservation, check your Quota Reservations again:

In [ ]:
all_quota_reservations = session.get(QUOTA_URL)
all_quota_reservations.raise_for_status()
all_quota_res_json = all_quota_reservations.json()

Status code: 200


In [103]:
all_quota_res_json

{'meta': {'args': {'limit': 50, 'offset': 0},
  'count': 2,
  'next': None,
  'prev': None},
 'results': [{'amount': 25.0,
   'aoi_ref': 'pl:features/my/cairo_areas-Kr7JYmq/1-owL5RyP',
   'collection_id': 'Kr7JYmq',
   'created_at': '2026-01-29T22:29:37.547929',
   'id': 1413740,
   'organization_id': 776979,
   'product_id': 290378,
   'state': 'reserved',
   'updated_at': '2026-01-29T22:29:37.547940',
   'url': 'https://api.planet.com/auth/v1/experimental/public/quota-reservations/1413740',
   'user_id': 820676},
  {'amount': 25.0,
   'aoi_ref': 'pl:features/my/cairo_areas-Kr7JYmq/0-gAbBZDk',
   'collection_id': 'Kr7JYmq',
   'created_at': '2026-01-29T22:39:19.249215',
   'id': 1413741,
   'organization_id': 776979,
   'product_id': 290378,
   'state': 'reserved',
   'updated_at': '2026-01-29T22:39:19.249215',
   'url': 'https://api.planet.com/auth/v1/experimental/public/quota-reservations/1413741',
   'user_id': 820676}]}

If you wanted to access a single quota reservation, you can do so using its ID.

In [ ]:
# For the example, we will just use the first result.

res_id = all_quota_res_json['results'][0]['id']

In [ ]:
SINGLE_QUOTA_RESERVATION = QUOTA_URL + str(res_id)
get_quota_res = session.get(SINGLE_QUOTA_RESERVATION)
get_quota_res.raise_for_status()
single_quota_res_json = get_quota_res.json()

Status code: 200


In [143]:
single_quota_res_json

{'amount': 25.0,
 'aoi_ref': 'pl:features/my/cairo_areas-Kr7JYmq/1-owL5RyP',
 'collection_id': 'Kr7JYmq',
 'created_at': '2026-01-29T22:29:37.547929',
 'id': 1413740,
 'organization_id': 776979,
 'product_id': 290378,
 'state': 'reserved',
 'updated_at': '2026-01-29T22:29:37.547940',
 'url': 'https://api.planet.com/auth/v1/experimental/public/quota-reservations/1413740',
 'user_id': 820676}

____

### Next Step: Create a workflow with the Features, Quota, and Subscriptions APIs all together!

Congratulations! You've completed the Quota API demo. You should now understand how the quota API is used to provision your quota for imagery, and you now have the tools to do so. We have created a full workflow to create features, provision quota, and create a subscription for you to follow if you wish, which can be found in our `workflows` notebooks in the [subscription_with_features](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/workflows/subscription_to_features/quota_subscription_from_features.ipynb) section.